<a href="https://colab.research.google.com/github/trefftzc/cis677/blob/main/NUMBA_CUDA_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A NUMBA/CUDA Tutorial

Based on https://numba.readthedocs.io/en/stable/cuda/kernels.html

The execution environment needs some special setups:

In [10]:
import os
os.environ['NUMBAPRO_LIBDEVICE'] = "/usr/local/cuda-12.5/nvvm/libdevice"
os.environ['NUMBAPRO_NVVM'] = "/usr/local/cuda-12.5/nvvm/lib64/libnvvm.so"

In [11]:
!uv pip install -q --system numba-cuda==0.4.0

# Introduction

CUDA has an execution model unlike the traditional sequential model used for programming CPUs. In CUDA, the code you write will be executed by multiple threads at once (often hundreds or thousands). Your solution will be modeled by defining a thread hierarchy of grid, blocks and threads.

Numba’s CUDA support exposes facilities to declare and manage this hierarchy of threads. The facilities are largely similar to those exposed by NVidia’s CUDA C language.

Numba also exposes three kinds of GPU memory: global device memory (the large, relatively slow off-chip memory that’s connected to the GPU itself), on-chip shared memory and local memory. For all but the simplest algorithms, it is important that you carefully consider how to use and access memory in order to minimize bandwidth requirements and contention.

# Kernel declaration

A kernel function is a GPU function that is meant to be called from CPU code (*). It gives it two fundamental characteristics:

kernels cannot explicitly return a value; all result data must be written to an array passed to the function (if computing a scalar, you will probably pass a one-element array);
kernels explicitly declare their thread hierarchy when called: i.e. the number of thread blocks and the number of threads per block (note that while a kernel is compiled once, it can be called multiple times with different block sizes or grid sizes).
At first sight, writing a CUDA kernel with Numba looks very much like writing a JIT function for the CPU:

In [12]:
from numba import config
config.CUDA_ENABLE_PYNVJITLINK = 1

In [13]:

from numba import cuda
@cuda.jit
def increment_by_one(an_array):
    """
    Increment all array elements by one.
    """
    # code elided here; read further for different implementations

# Kernel invocation

A kernel is typically launched in the following way:

In [14]:
import numpy as np
threadsperblock = 32
an_array = np.arange(2048)
blockspergrid = (an_array.size + (threadsperblock - 1)) // threadsperblock
increment_by_one[blockspergrid, threadsperblock](an_array)

/usr/local/lib/python3.12/dist-packages/numba_cuda/numba/cuda/dispatcher.py:680: NumbaPerformanceWarning: Grid size 64 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/usr/local/lib/python3.12/dist-packages/numba_cuda/numba/cuda/cudadrv/devicearray.py:940: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.


We notice two steps here:

Instantiate the kernel proper, by specifying a number of blocks (or “blocks per grid”), and a number of threads per block. The product of the two will give the total number of threads launched. Kernel instantiation is done by taking the compiled kernel function (here increment_by_one) and indexing it with a tuple of integers.
Running the kernel, by passing it the input array (and any separate output arrays if necessary). Kernels run asynchronously: launches queue their execution on the device and then return immediately. You can use cuda.synchronize() to wait for all previous kernel launches to finish executing.

Note

Passing an array that resides in host memory will implicitly cause a copy back to the host, which will be synchronous. In this case, the kernel launch will not return until the data is copied back, and therefore appears to execute synchronously.

# Choosing the block size

It might seem curious to have a two-level hierarchy when declaring the number of threads needed by a kernel. The block size (i.e. number of threads per block) is often crucial:

On the software side, the block size determines how many threads share a given area of shared memory.
On the hardware side, the block size must be large enough for full occupation of execution units; recommendations can be found in the CUDA C Programming Guide.

# Multi-dimensional blocks and grids

To help deal with multi-dimensional arrays, CUDA allows you to specify multi-dimensional blocks and grids. In the example above, you could make blockspergrid and threadsperblock tuples of one, two or three integers. Compared to 1D declarations of equivalent sizes, this doesn’t change anything to the efficiency or behaviour of generated code, but can help you write your algorithms in a more natural way.

# Thread positioning

When running a kernel, the kernel function’s code is executed by every thread once. It therefore has to know which thread it is in, in order to know which array element(s) it is responsible for (complex algorithms may define more complex responsibilities, but the underlying principle is the same).

One way is for the thread to determine its position in the grid and block and manually compute the corresponding array position:

In [15]:
from numba import cuda
@cuda.jit
def increment_by_one(an_array):
    # Thread id in a 1D block
    tx = cuda.threadIdx.x
    # Block id in a 1D grid
    ty = cuda.blockIdx.x
    # Block width, i.e. number of threads per block
    bw = cuda.blockDim.x
    # Compute flattened index inside the array
    pos = tx + ty * bw
    if pos < an_array.size:  # Check array boundaries
        an_array[pos] += 1

Unless you are sure the block size and grid size is a divisor of your array size, you must check boundaries as shown above.
threadIdx, blockIdx, blockDim and gridDim are special objects provided by the CUDA backend for the sole purpose of knowing the geometry of the thread hierarchy and the position of the current thread within that geometry.

These objects can be 1D, 2D or 3D, depending on how the kernel was invoked. To access the value at each dimension, use the x, y and z attributes of these objects, respectively.

numba.cuda.threadIdx

The thread indices in the current thread block. For 1D blocks, the index (given by the x attribute) is an integer spanning the range from 0 inclusive to numba.cuda.blockDim exclusive. A similar rule exists for each dimension when more than one dimension is used.

numba.cuda.blockDim

The shape of the block of threads, as declared when instantiating the kernel. This value is the same for all threads in a given kernel, even if they belong to different blocks (i.e. each block is “full”).
numba.cuda.blockIdx
The block indices in the grid of threads launched a kernel. For a 1D grid, the index (given by the x attribute) is an integer spanning the range from 0 inclusive to numba.cuda.gridDim exclusive. A similar rule exists for each dimension when more than one dimension is used.

numba.cuda.gridDim

The shape of the grid of blocks, i.e. the total number of blocks launched by this kernel invocation, as declared when instantiating the kernel.

# Absolute positions

Simple algorithms will tend to always use thread indices in the same way as shown in the example above. Numba provides additional facilities to automate such calculations:

numba.cuda.grid(ndim)
Return the absolute position of the current thread in the entire grid of blocks. ndim should correspond to the number of dimensions declared when instantiating the kernel. If ndim is 1, a single integer is returned. If ndim is 2 or 3, a tuple of the given number of integers is returned.
numba.cuda.gridsize(ndim)
Return the absolute size (or shape) in threads of the entire grid of blocks. ndim has the same meaning as in grid() above.

With these functions, the incrementation example can become:

In [16]:
@cuda.jit
def increment_by_one(an_array):
    pos = cuda.grid(1)
    if pos < an_array.size:
        an_array[pos] += 1

The same example for a 2D array and grid of threads would be:

In [17]:
@cuda.jit
def increment_a_2D_array(an_array):
    x, y = cuda.grid(2)
    if x < an_array.shape[0] and y < an_array.shape[1]:
       an_array[x, y] += 1

Note the grid computation when instantiating the kernel must still be done manually, for example:

In [18]:
import math
import numpy as np
an_array = np.arange(1024).reshape(32,32)
threadsperblock = (16, 16)
blockspergrid_x = math.ceil(an_array.shape[0] / threadsperblock[0])
blockspergrid_y = math.ceil(an_array.shape[1] / threadsperblock[1])
blockspergrid = (blockspergrid_x, blockspergrid_y)
increment_a_2D_array[blockspergrid, threadsperblock](an_array)

/usr/local/lib/python3.12/dist-packages/numba_cuda/numba/cuda/dispatcher.py:680: NumbaPerformanceWarning: Grid size 4 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/usr/local/lib/python3.12/dist-packages/numba_cuda/numba/cuda/cudadrv/devicearray.py:940: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.


# Memory management

# Data transfer

Even though Numba can automatically transfer NumPy arrays to the device, it can only do so conservatively by always transferring device memory back to the host when a kernel finishes. To avoid the unnecessary transfer for read-only arrays, you can use the following APIs to manually control the transfer:

numba.cuda.device_array(shape, dtype=np.float64, strides=None, order='C', stream=0)

Allocate an empty device ndarray. Similar to numpy.empty().

numba.cuda.device_array_like(ary, stream=0)

Call device_array() with information from the array.

numba.cuda.to_device(obj, stream=0, copy=True, to=None)

Allocate and transfer a numpy ndarray or structured scalar to the device.

To copy host->device a numpy array:

In [20]:
ary = np.arange(10)
d_ary = cuda.to_device(ary)

The resulting d_ary is a DeviceNDArray.

To copy device->host:

In [21]:
hary = d_ary.copy_to_host()

To copy device->host to an existing array:

In [22]:
ary = np.empty(shape=d_ary.shape, dtype=d_ary.dtype)
d_ary.copy_to_host(ary)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

# Device arrays

Device array references have the following methods. These methods are to be called in host code, not within CUDA-jitted functions.

class numba.cuda.cudadrv.devicearray.DeviceNDArray(shape, strides, dtype, stream=0, gpu_data=None)

An on-GPU array type

copy_to_host(ary=None, stream=0)

Copy self to ary or create a new Numpy ndarray if ary is None.

Always returns the host array.

Example:

In [27]:
import numpy as np
from numba import cuda

@cuda.jit
def my_kernel(an_array):
    x = cuda.grid(1)
    if x < an_array.shape[0]:
       an_array[x] += 1

arr = np.arange(10000)
d_arr = cuda.to_device(arr)

my_kernel[100, 100](d_arr)

result_array = d_arr.copy_to_host()

/usr/local/lib/python3.12/dist-packages/numba_cuda/numba/cuda/dispatcher.py:680: NumbaPerformanceWarning: Grid size 100 will likely result in GPU under-utilization due to low occupancy.
  and not config.DISABLE_PERFORMANCE_WARNINGS


# Examples


# Vector Addition

This example uses Numba to create on-device arrays and a vector addition kernel; it is a warmup for learning how to write GPU kernels using Numba. We’ll begin with some required imports:



In [28]:
import numpy as np
from numba import cuda

The following function is the kernel. Note that it is defined in terms of Python variables with unspecified types. When the kernel is launched, Numba will examine the types of the arguments that are passed at runtime and generate a CUDA kernel specialized for them.

Note that Numba kernels do not return values and must write any output into arrays passed in as parameters (this is similar to the requirement that CUDA C/C++ kernels have void return type). Here we pass in c for the results to be written into.

In [29]:
@cuda.jit
def f(a, b, c):
    # like threadIdx.x + (blockIdx.x * blockDim.x)
    tid = cuda.grid(1)
    size = len(c)

    if tid < size:
        c[tid] = a[tid] + b[tid]

cuda.to_device() can be used create device-side copies of arrays. cuda.device_array_like() creates an uninitialized array of the same shape and type as an existing array. Here we transfer two vectors and create an empty vector to hold our results:

In [30]:
N = 100000
a = cuda.to_device(np.random.random(N))
b = cuda.to_device(np.random.random(N))
c = cuda.device_array_like(a)

A call to forall() generates an appropriate launch configuration with a 1D grid (see Kernel invocation) for a given data size and is often the simplest way of launching a kernel:

In [31]:
f.forall(len(a))(a, b, c)
print(c.copy_to_host())

[1.17708359 1.40638864 0.52871927 ... 0.98011679 0.42299897 1.64504293]


/usr/local/lib/python3.12/dist-packages/numba_cuda/numba/cuda/dispatcher.py:680: NumbaPerformanceWarning: Grid size 98 will likely result in GPU under-utilization due to low occupancy.
  and not config.DISABLE_PERFORMANCE_WARNINGS


One can also configure the grid manually using the subscripting syntax. The following example launches a grid with sufficient threads to operate on every vector element:

In [32]:
# Enough threads per block for several warps per block
nthreads = 256
# Enough blocks to cover the entire vector depending on its length
nblocks = (len(a) // nthreads) + 1
f[nblocks, nthreads](a, b, c)
print(c.copy_to_host())

[1.17708359 1.40638864 0.52871927 ... 0.98011679 0.42299897 1.64504293]


# 1D Heat Equation

This example solves Laplace’s equation in one dimension for a certain set of initial conditions and boundary conditions. A full discussion of Laplace’s equation is out of scope for this documentation, but it will suffice to say that it describes how heat propagates through an object over time. It works by discretizing the problem in two ways:

The domain is partitioned into a mesh of points that each have an individual temperature.
Time is partitioned into discrete intervals that are advanced forward sequentially.
Then, the following assumption is applied: The temperature of a point after some interval has passed is some weighted average of the temperature of the points that are directly adjacent to it. Intuitively, if all the points in the domain are very hot and a single point in the middle is very cold, as time passes, the hot points will cause the cold one to heat up and the cold point will cause the surrounding hot pieces to cool slightly. Simply put, the heat spreads throughout the object.

We can implement this simulation using a Numba kernel. Let’s start simple by assuming we have a one dimensional object which we’ll represent with an array of values. The position of the element in the array is the position of a point within the object, and the value of the element represents the temperature.

In [33]:
import numpy as np
from numba import cuda

In [40]:
# Use an odd problem size.
 # This is so there can be an element truly in the "middle" for symmetry.
size = 1001
data = np.zeros(size)

 # Middle element is made very hot
data[500] = 10000
buf_0 = cuda.to_device(data)

# This extra array is used for synchronization purposes
buf_1 = cuda.device_array_like(buf_0)

niter = 10000

In our kernel each thread will be responsible for managing the temperature update for a single element in a loop over the desired number of timesteps. The kernel is below. Note the use of cooperative group synchronization and the use of two buffers swapped at each iteration to avoid race conditions. See numba.cuda.cg.this_grid() for details.

In [37]:
@cuda.jit
def solve_heat_equation(buf_0, buf_1, timesteps, k):
     i = cuda.grid(1)

     # Don't continue if our index is outside the domain
     if i >= len(buf_0):
         return

     # Prepare to do a grid-wide synchronization later
     grid = cuda.cg.this_grid()

     for step in range(timesteps):
        # Select the buffer from the previous timestep
        if (step % 2) == 0:
            data = buf_0
            next_data = buf_1
        else:
            data = buf_1
            next_data = buf_0

        # Get the current temperature associated with this point
        curr_temp = data[i]

        # Apply formula from finite difference equation
        if i == 0:
            # Left wall is held at T = 0
            next_temp = curr_temp + k * (data[i + 1] - (2 * curr_temp))
        elif i == len(data) - 1:
            # Right wall is held at T = 0
            next_temp = curr_temp + k * (data[i - 1] - (2 * curr_temp))
        else:
            # Interior points are a weighted average of their neighbors
            next_temp = curr_temp + k * (
                data[i - 1] - (2 * curr_temp) + data[i + 1]
            )

        # Write new value to the next buffer
        next_data[i] = next_temp

        # Wait for every thread to write before moving on
        grid.sync()

Calling the kernel:

In [41]:
solve_heat_equation.forall(len(data))(
    buf_0, buf_1, niter, 0.25
)

/usr/local/lib/python3.12/dist-packages/numba_cuda/numba/cuda/dispatcher.py:680: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  and not config.DISABLE_PERFORMANCE_WARNINGS


Plotting the final data shows an arc that is highest where the object was hot initially and gradually sloping down to zero towards the edges where the temperature is fixed at zero. In the limit of infinite time, the arc will flatten out completely.